# 🧬 **Yapay Zeka Destekli İlaç Keşfi: ABL1, FLT3, BTK, KIT, BCL2, JAK2 Hedefli Molekül Üretimi**

Bu proje, **Kronik Miyeloid Lösemi (CML)** tedavisinde kritik bir hedef olan **ABL1, FLT3, BTK, KIT, BCL2, JAK2** proteinini baskılayabilecek (inhibitör) yeni ilaç aday moleküllerini **Generative AI (Üretken Yapay Zeka)** kullanarak tasarlamayı amaçlamaktadır.

---

### 🚀 **Proje Akışı:**
1.  **Biyoinformatik:** NCBI'dan hedef protein sekansının çekilmesi.
2.  **Veri Madenciliği:** ChEMBL veritabanından bu hedefe etki eden ilaçların bulunması.
3.  **Keminformatik:** RDKit ile moleküllerin temizlenmesi ve SMILES formatına çevrilmesi.
4.  **Deep Learning:** PyTorch ile LSTM tabanlı bir dil modeli eğitilmesi.
5.  **De Novo Tasarım:** Eğitilen model ile tamamen yeni moleküllerin üretilmesi.

### 🛠️ **Kullanılan Kütüphaneler:**
<span style="background-color: #f1c40f; color: black; padding: 4px 8px; border-radius: 4px; font-weight: bold; margin-right:5px;">BioPython</span>
<span style="background-color: #008080; color: white; padding: 4px 8px; border-radius: 4px; font-weight: bold; margin-right:5px;">ChEMBL Web Resource</span>
<span style="background-color: #3498db; color: white; padding: 4px 8px; border-radius: 4px; font-weight: bold; margin-right:5px;">RDKit</span>
<span style="background-color: #e74c3c; color: white; padding: 4px 8px; border-radius: 4px; font-weight: bold; margin-right:5px;">PyTorch</span>

In [1]:
!pip install biopython
!pip install chembl-webresource-client
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 54.9 MB/s eta 0:00:00


In [4]:
# Gerekli kütüphaneler: biopython, chembl_webresource_client, rdkit
from Bio import Entrez, SeqIO
import time
from chembl_webresource_client.new_client import new_client
import pandas as pd

## 1️⃣ **Biyoinformatik Veri Toplama: ABL1 Proteini**

<div style="background-color: #e8f6f3; padding: 15px; border-left: 5px solid #1abc9c; border-radius: 5px;">
<strong>Hedef:</strong> NCBI veritabanını kullanarak Lösemi ile ilişkili <code>NP_005148</code> kodlu ABL1 proteininin FASTA sekansını çekmek.
</div>

In [6]:
target_genes = ['ABL1', 'FLT3', 'BTK', 'KIT', 'BCL2', 'JAK2']

# NCBI için Gen ID'leri (FASTA çekmek istersen diye)
# Bu ID'ler insan (Homo sapiens) protein referanslarıdır.
ncbi_protein_ids = {
    'ABL1': 'NP_005148',
    'FLT3': 'NP_004110',
    'BTK':  'NP_000052',
    'KIT':  'NP_000213',
    'BCL2': 'NP_000624',
    'JAK2': 'NP_004963'
}

Entrez.email = "240542017@firat.edu.com" # Burayı kendi mailinle doldur

# --- KISIM 1: İLAÇ VERİLERİNİ (SMILES) TOPLAMA ---
all_smiles_data = []

print("İlaç verileri ChEMBL üzerinden çekiliyor...")

for gene in target_genes:
    print(f"  -> {gene} taranıyor...")

    # 1. Hedef ID'sini bul
    target = new_client.target
    target_query = target.filter(target_synonym__icontains=gene).filter(target_organism__icontains='Homo sapiens')

    if not target_query:
        print(f"     UYARI: {gene} ChEMBL'de bulunamadı.")
        continue

    target_id = target_query[0]['target_chembl_id']

    # 2. Bu hedefe ait aktiviteleri (IC50) bul
    activity = new_client.activity
    res = activity.filter(target_chembl_id=target_id).filter(standard_type="IC50")

    # 3. SMILES kodlarını listeye ekle
    # Her kaydın yanına hangi gen için çekildiğini de ekliyoruz (meta-data)
    count = 0
    for item in res:
        if item['canonical_smiles']:
            all_smiles_data.append({
                'Gene': gene,
                'SMILES': item['canonical_smiles'],
                'IC50': item['value'] # İlacın gücü (opsiyonel, filtrelemede kullanılabilir)
            })
            count += 1
    print(f"     {count} molekül eklendi.")

# DataFrame oluştur
df_all = pd.DataFrame(all_smiles_data)
print(f"\nTOPLAM HAM VERİ: {len(df_all)} satır.")

# --- KISIM 2: FASTA (PROTEİN DİZİLERİ) TOPLAMA ---
print("\nProtein dizileri NCBI üzerinden çekiliyor...")

with open("leukemia_targets_combined.fasta", "w") as fasta_file:
    for gene, ref_id in ncbi_protein_ids.items():
        try:
            handle = Entrez.efetch(db="protein", id=ref_id, rettype="fasta", retmode="text")
            seq_data = handle.read()
            fasta_file.write(seq_data) # Dosyaya ekle (append mantığı)
            print(f"  -> {gene} ({ref_id}) eklendi.")
        except Exception as e:
            print(f"  -> HATA: {gene} çekilemedi. {e}")

print("Tüm proteinler 'leukemia_targets_combined.fasta' dosyasına kaydedildi.")

İlaç verileri ChEMBL üzerinden çekiliyor...
  -> ABL1 taranıyor...
     5287 molekül eklendi.
  -> FLT3 taranıyor...
     7094 molekül eklendi.
  -> BTK taranıyor...
     21606 molekül eklendi.
  -> KIT taranıyor...
     4363 molekül eklendi.
  -> BCL2 taranıyor...
     65 molekül eklendi.
  -> JAK2 taranıyor...
     19204 molekül eklendi.

TOPLAM HAM VERİ: 57619 satır.

Protein dizileri NCBI üzerinden çekiliyor...
  -> ABL1 (NP_005148) eklendi.
  -> FLT3 (NP_004110) eklendi.
  -> BTK (NP_000052) eklendi.
  -> KIT (NP_000213) eklendi.
  -> BCL2 (NP_000624) eklendi.
  -> JAK2 (NP_004963) eklendi.
Tüm proteinler 'leukemia_targets_combined.fasta' dosyasına kaydedildi.


In [7]:
from rdkit import Chem

def canonicalize_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return Chem.MolToSmiles(mol, canonical=True)
        else:
            return None
    except:
        return None

print("Veri temizleniyor ve kanonikleştiriliyor...")

# 1. Canonical SMILES oluştur
df_all['Canonical_SMILES'] = df_all['SMILES'].apply(canonicalize_smiles)

# 2. Geçersiz olanları at
df_all = df_all.dropna(subset=['Canonical_SMILES'])

# 3. KOPYALARI SİL (En önemli adım)
# Aynı Canonical_SMILES'a sahip satırları sil, sadece birini tut.
df_clean_all = df_all.drop_duplicates(subset=['Canonical_SMILES'])

print(f"Temizlendikten sonraki EŞSİZ molekül sayısı: {len(df_clean_all)}")

# 4. Kaydet
df_clean_all.to_csv('leukemia_multitarget_dataset.csv', index=False)

Veri temizleniyor ve kanonikleştiriliyor...
Temizlendikten sonraki EŞSİZ molekül sayısı: 32437


In [8]:
import re

def tokenize_smiles(smiles):
    pattern = r"(\[[^\[\]]{1,6}\])"
    tokens = re.split(pattern, smiles)
    result = []
    for token in tokens:
        if token.startswith('['):
            result.append(token)
        else:
            result.extend(list(token))
    return result

# Example usage
print(tokenize_smiles('CC(=O)OC1=CC=CC=C1C(=O)O'))

['C', 'C', '(', '=', 'O', ')', 'O', 'C', '1', '=', 'C', 'C', '=', 'C', 'C', '=', 'C', '1', 'C', '(', '=', 'O', ')', 'O']


In [9]:
# Tüm veri setindeki SMILES'ları tarayıp benzersiz tokenleri bulalım
all_smiles = df_clean_all['Canonical_SMILES'].tolist()
unique_tokens = set()

for smile in all_smiles:
    tokens = tokenize_smiles(smile)
    unique_tokens.update(tokens)

# Özel tokenleri ekleyelim
# <PAD>: Dolgu (0), <SOS>: Başlangıç, <EOS>: Bitiş
vocab = sorted(list(unique_tokens))
vocab = ['<PAD>', '<SOS>', '<EOS>'] + vocab

# Token -> ID (stoi) ve ID -> Token (itos) sözlüklerini kuralım
stoi = { token:i for i, token in enumerate(vocab) }
itos = { i:token for i, token in enumerate(vocab) }

print(f"Toplam Benzersiz Token Sayısı (Vocab Size): {len(vocab)}")
print("Örnek Token ID'leri:", list(stoi.items())[:10])

Toplam Benzersiz Token Sayısı (Vocab Size): 57
Örnek Token ID'leri: [('<PAD>', 0), ('<SOS>', 1), ('<EOS>', 2), ('#', 3), ('(', 4), (')', 5), ('-', 6), ('.', 7), ('/', 8), ('1', 9)]


In [10]:
import torch
import numpy as np

# Maksimum molekül uzunluğunu bulalım (Padding için gerekli)
max_len = max([len(tokenize_smiles(s)) for s in all_smiles]) + 2 # +2 SOS ve EOS için
print(f"En uzun molekül uzunluğu: {max_len}")

def smile_to_vector(smile, vocab_stoi, max_len):
    tokens = tokenize_smiles(smile)
    # Başına <SOS>, sonuna <EOS> ekle
    vector = [vocab_stoi['<SOS>']] + [vocab_stoi[t] for t in tokens] + [vocab_stoi['<EOS>']]

    # Padding (Geri kalan kısmı <PAD> yani 0 ile doldur)
    padding = [vocab_stoi['<PAD>']] * (max_len - len(vector))
    return vector + padding

# Tüm veri setini vektörleştirelim
X_data = []
for smile in all_smiles:
    vec = smile_to_vector(smile, stoi, max_len)
    X_data.append(vec)

# Tensor'a çevirelim (PyTorch formatı)
X_tensor = torch.tensor(X_data, dtype=torch.long)
print(f"Veri Seti Boyutu: {X_tensor.shape}") # (2851, max_len) olmalı

En uzun molekül uzunluğu: 210
Veri Seti Boyutu: torch.Size([32437, 210])


In [11]:
from torch.utils.data import Dataset, DataLoader

class DrugDataset(Dataset):
    def __init__(self, data_tensor):
        self.data = data_tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Girdi (Input): Molekülün başından sonuna kadar (son karakter hariç)
        # Hedef (Target): Molekülün 2. karakterinden sonuna kadar (bir adım kaydırılmış)
        # Model 'A'yı görünce 'B'yi tahmin etmeye çalışacak.

        full_seq = self.data[idx]
        x = full_seq[:-1] # Girdi
        y = full_seq[1:]  # Hedef (Sonraki karakter)
        return x, y

# Dataset ve DataLoader oluşturma
batch_size = 32 # T4 GPU için uygun bir boyut
dataset = DrugDataset(X_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Test edelim
sample_x, sample_y = next(iter(dataloader))
print("Girdi (X) Boyutu:", sample_x.shape)
print("Hedef (Y) Boyutu:", sample_y.shape)

Girdi (X) Boyutu: torch.Size([32, 209])
Hedef (Y) Boyutu: torch.Size([32, 209])


## 5️⃣ **LSTM Model Mimarisi (Generative AI)**
<div style="background-color: #fadbd8; padding: 15px; border-left: 5px solid #e74c3c; border-radius: 5px;">
Modelimiz, bir önceki karaktere bakarak bir sonraki karakteri tahmin etmeye çalışan bir <strong>RNN (LSTM)</strong> ağıdır.
</div>

* **Embedding Layer:** Karakterleri yoğun (dense) vektörlere çevirir.
* **LSTM Layer:** Dizi bilgisini ve uzun süreli bağımlılıkları öğrenir.
* **Linear Layer:** Sonuçları tekrar Vocabulary boyutuna indirger.

In [ ]:
import torch.nn as nn

class MoleculeGenerator(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(MoleculeGenerator, self).__init__()

        # 1. Embedding: Sayıları vektörlere çevirir (Örn: 5 -> [0.1, -0.5, ...])
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # 2. LSTM: Dizideki ilişkiyi öğrenir
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)

        # 3. Linear: Sonraki karakterin olasılığını hesaplar
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        # x shape: (batch_size, seq_len)
        embed = self.embedding(x)

        # out shape: (batch_size, seq_len, hidden_size)
        out, (ht, ct) = self.lstm(embed, hidden)

        # output shape: (batch_size, seq_len, vocab_size)
        output = self.fc(out)
        return output, (ht, ct)

# Model Parametreleri
VOCAB_SIZE = len(vocab)
EMBED_SIZE = 128   # Her token ne kadar detaylı temsil edilsin?
HIDDEN_SIZE = 256  # Modelin hafıza kapasitesi
NUM_LAYERS = 2     # Kaç katmanlı LSTM?

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MoleculeGenerator(VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS).to(device)

print(model)
print(f"Model {device} üzerinde çalışacak.")

### 🏋️‍♂️ Model Eğitimi
Model, ABL1 inhibitörlerinin SMILES yapılarını öğrenmek üzere eğitiliyor. Her 2 epoch'ta bir örnek üretim yaparak gelişimi gözlemleyeceğiz.

In [ ]:
import time
import torch.optim as optim
import torch.nn as nn

# Hiperparametreler ve Ayarlar
LEARNING_RATE = 0.001
EPOCHS = 1000

# Eğer daha önce tanımlanmadıysa tekrar tanımlayalım
criterion = nn.CrossEntropyLoss(ignore_index=stoi['<PAD>'])
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

def train(model, dataloader, epochs):
    model.train() # Başlangıçta eğitim modu
    loss_history = []
    print(f"Eğitim Başlıyor... ({device})")

    for epoch in range(epochs):
        start_time = time.time()
        total_loss = 0

        for batch_idx, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            output, _ = model(x)

            loss = criterion(output.view(-1, VOCAB_SIZE), y.view(-1))

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        loss_history.append(avg_loss)
        elapsed = time.time() - start_time

        if (epoch+1) % 2 == 0:
            print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Süre: {elapsed:.1f}sn")

            # Her 2 epochta bir örnek üretip gelişimini görelim (Sıklığı artırdım)
            if (epoch+1) % 2 == 0:
                print("   -> Örnek üretiliyor...")

                # 1. Örnek Üret (Bu işlem model.eval() yapar)
                test_smiles = generate_molecule(model, start_str="C", temperature=0.8)
                print(f"   -> Sonuç: {test_smiles}")

                # 2. KRİTİK DÜZELTME: Modeli tekrar eğitim moduna al!
                model.train()
                # ^^^ BU SATIR EKSİKTİ ^^^

    return loss_history

# Eğitimi tekrar başlat (Modelin kaldığı yerden devam eder ama temiz olsun dersen modeli tekrar baştan tanımlayabilirsin)
loss_history = train(model, dataloader, EPOCHS)

# Modeli Kaydet
torch.save(model.state_dict(), "oncomind_model.pth")
print("Model başarıyla kaydedildi!")

## 6️⃣ **Validasyon ve İlaç Uygunluk Testi**
Üretilen moleküllerin geçerli kimyasal yapılar olup olmadığını ve ilaç olmaya adaylık skorlarını (QED) kontrol ediyoruz.

In [ ]:
from  rdkit  import  Chem
from  rdkit.Chem  import  Draw

mol  =  Chem.MolFromSmiles("CC(C)(C)C(=O)N1Cc2c(NC(=O)c3cc(F)cc(F)c3)n[nH]c2C1(C)C")
Draw.MolToImage(mol)

In [ ]:
def generate_molecule(model, start_str="C", max_length=100, temperature=1.0):
    model.eval() # Değerlendirme modu

    # Başlangıç karakterini vektöre çevir
    input_seq = [stoi['<SOS>']] + [stoi[c] for c in start_str]
    input_tensor = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)

    generated_str = start_str
    hidden = None

    with torch.no_grad():
        for _ in range(max_length):
            output, hidden = model(input_tensor, hidden)

            # Son karakterin tahminini al
            last_token_logits = output[0, -1, :] / temperature
            probs = torch.nn.functional.softmax(last_token_logits, dim=0)

            # Olasılıklara göre bir sonraki karakteri seç
            next_token_idx = torch.multinomial(probs, 1).item()

            if next_token_idx == stoi['<EOS>']:
                break

            next_char = itos[next_token_idx]
            generated_str += next_char

            # Yeni girdiyi güncelle
            input_tensor = torch.tensor([[next_token_idx]], dtype=torch.long).to(device)

    return generated_str

generated_molecule = generate_molecule(model, start_str="C", temperature=0.8)
print("Üretilen Örnek:", generated_molecule)

In [ ]:
# Modelin ürettiği çıktıyı analiz etme ve eleme kısmı (Organ yetmezliği kısıtı)
from rdkit import Chem
from rdkit.Chem import Descriptors, QED

def check_drug_quality(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return False

    # 1. QED (Quantitative Estimation of Drug-likeness)
    # Bu skor ilacın genel olarak vücutta emilip emilemeyeceğini tahmin eder.
    qed_score = QED.qed(mol)

    # 2. Moleküler Ağırlık (Çok ağırsa böbreklerden atılamaz)
    mw = Descriptors.MolWt(mol)

    # Filtre: İlaç benzeri olsun VE çok ağır olmasın (Basit bir toksisite önlemi)
    if qed_score > 0.5 and mw < 500:
        return True, f"Molekül uygun. QED: {qed_score:.2f}, MW: {mw:.1f}"
    else:
        return False, "Uygunsuz."

# Örnek Kullanım
generated_molecule = "CC(C)(C)C(=O)N1Cc2c(NC(=O)c3cc(F)cc(F)c3)n[nH]c2C1(C)C" # (Imatinib benzeri bir yapı)
is_good, reason = check_drug_quality(generated_molecule)
print(reason)